# Classifying Images with Deep Convolutional Neural Networks 

## Determining the size of convolution output 

In [1]:
import numpy as np

Discrete Convolution in 1D

In [2]:
def conv1d(x, w, p = 0, s = 1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
print('Conv1d Implementation: ', 
      conv1d(x, w, p=2, s=1))
print('Numpy results: ',
      np.convolve(x, w, mode = 'same'))

Conv1d Implementation:  [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy results:  [ 5 14 16 26 24 34 19 22]


Discrete Convolution in 2D

In [3]:
import scipy.signal

In [4]:
def conv2d(X, W, p = (0, 0), s = (1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape = (n1, n2))
    X_padded[p[0]:p[0] + X_orig.shape[0],
             p[1]:p[1] + X_orig.shape[1]] = X_orig
    res = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i + W_rot.shape[0], j:j + W_rot.shape[1]]
            res[-1].append(np.sum(X_sub*W_rot))
    return np.array(res)

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1 , 2,0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]
print('Conv2d Implementation: \n', 
      conv2d(X, W, p=(1,1), s=(1,1)))
print('Scipy results: \n',
      scipy.signal.convolve2d(X, W, mode = 'same'))

Conv2d Implementation: 
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
Scipy results: 
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


## Working with multiple inputs or color channels 

In [5]:
import imageio

In [6]:
try:
    img = imageio.imread('./example-image.png', pilmode = 'RGB')
except AttributeError:
    s = ("imageio.imread requires Python's image library PIL"
         " You can satisfy this requirement by installing the"
         " userfriendly fork PILLOW via `pip install pillow`.")
    raise AttributeError(s)

print('Image shape:', img.shape)
print('Number of channels: ', img.shape[2])
print('Image data type: ', img.dtype)

print(img[100:102, 100:102, :])

Image shape: (252, 221, 3)
Number of channels:  3
Image data type:  uint8
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


# Implementing deep convolutional neural network using Tensorflow 

## Loading and preprocessing the data 

In [8]:
import struct
import numpy as np
import os

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                                % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels


X_data, y_data = load_mnist('./', kind='train')
print('Rows: %d,  Columns: %d' % (X_data.shape[0], X_data.shape[1]))
X_test, y_test = load_mnist('./', kind='t10k')
print('Rows: %d,  Columns: %d' % (X_test.shape[0], X_test.shape[1]))

X_train, y_train = X_data[:50000,:], y_data[:50000]
X_valid, y_valid = X_data[50000:,:], y_data[50000:]

print('Training:   ', X_train.shape, y_train.shape)
print('Validation: ', X_valid.shape, y_valid.shape)
print('Test Set:   ', X_test.shape, y_test.shape)

Rows: 60000,  Columns: 784
Rows: 10000,  Columns: 784
Training:    (50000, 784) (50000,)
Validation:  (10000, 784) (10000,)
Test Set:    (10000, 784) (10000,)


In [9]:
def batch_generator(X, y, batch_size=64, 
                    shuffle=False, random_seed=None):
    
    idx = np.arange(y.shape[0])
    
    if shuffle:
        rng = np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X = X[idx]
        y = y[idx]
    
    for i in range(0, X.shape[0], batch_size):
        yield (X[i:i+batch_size, :], y[i:i+batch_size])

In [10]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_data, y_data, X_train, X_valid, X_test

## Implementing CNN in low level Tensorflow API 

In [12]:
import tensorflow as tf
import numpy as np

In [14]:
## wrapper functions

def conv_layer(input_tensor, name,
               kernel_size, n_output_channels,
               padding_mode = 'SAME', strides = (1, 1, 1, 1)):
    with tf.variable_scope(name):
        ## get n_input_channels:
        ##   input tensor shape:
        ##   [batch x width x height x channels_in]
        input_shape = input_tensor.get_shape().as_list()
        n_input_channels = input_shape[-1]
        
        weights_shape = (list(kernel_size) + 
                         [n_input_channels, n_output_channels])
        
        weights = tf.get_variable(name = '_weights',
                                  shape = weights_shape)
        print(weights)
        biases = tf.get_variable(name = '_biases',
                                 initializer = tf.zeros(shape = n_output_channels))
        print(biases)
        conv = tf.nn.conv2d(input = input_tensor,
                            filter = weights,
                            strides = strides,
                            padding = padding_mode)
        print(conv)
        conv = tf.nn.bias_add(conv, biases, name = 'net_pre-activation')
        print(conv)
        conv = tf.nn.relu(conv, name = 'activation')
        print(conv)
        
        return conv
    
## testing
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    conv_layer(x, name='convtest', kernel_size=(3, 3), n_output_channels=32)
    
del g, x

<tf.Variable 'convtest/_weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
<tf.Variable 'convtest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("convtest/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(?, 28, 28, 32), dtype=float32)


In [17]:
def fc_layer(input_tensor, name,
             n_output_units, activation_fn = None):
    with tf.variable_scope(name):
        input_shape = input_tensor.get_shape().as_list()[1:]
        n_input_units = np.prod(input_shape)
        if len(input_shape) > 1:
            input_tensor = tf.reshape(input_tensor,
                                      shape = (-1, n_input_units))
            
        weights_shape = [n_input_units, n_output_units]
        weights = tf.get_variable(name = '_weights',
                                  shape = weights_shape)
        print(weights)
        biases = tf.get_variable(name = '_biases',
                                 initializer = tf.zeros(shape = n_output_units))
        print(biases)
        layer = tf.matmul(input_tensor, weights)
        print(layer)
        layer = tf.nn.bias_add(layer, biases, name = 'net_pre-activation')
        print(layer)
        if activation_fn is None:
            return layer
        
        layer = activation_fn(layer, name = 'activation')
        print(layer)
        return layer
    
## testing
## testing:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, 
                       shape=[None, 28, 28, 1])
    fc_layer(x, name='fctest', n_output_units=32, 
             activation_fn=tf.nn.relu)
    
del g, x

<tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
<tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Tensor("fctest/net_pre-activation:0", shape=(?, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [21]:
def build_cnn(learning_rate = 1e-4):
    ## Placeholders for X and y
    tf_x = tf.placeholder(tf.float32, shape = [None, 784],
                          name = 'tf_x')
    tf_y = tf.placeholder(tf.int32, shape = [None],
                          name = 'tf_y')
    
    ## reshape X to 4D tensor:
    ## [batchsize, width, height, 1]
    tf_x_image = tf.reshape(tf_x, shape = [-1, 28, 28, 1],
                            name = 'tf_x_reshaped')
    ## One-hot encoding
    tf_y_onehot = tf.one_hot(indices = tf_y,
                             depth = 10,
                             dtype = tf.float32,
                             name = 'tf_y_onehot')
    
    ## 1st layer: Conv1
    print('\nBuilding 1st layer: ')
    h1 = conv_layer(tf_x_image, name = 'conv1',
                    kernel_size = (5, 5),
                    padding_mode = 'VALID',
                    n_output_channels = 32)
    ## MaxPooling
    h1_pool = tf.nn.max_pool(h1,
                         ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1],
                         padding = 'SAME')
    ## 2n layer: Conv_2
    print('\nBuilding 2nd layer: ')
    h2 = conv_layer(h1_pool, name = 'conv_2', 
                    kernel_size = (5,5), 
                    padding_mode = 'VALID',
                    n_output_channels = 64)
    ## MaxPooling 
    h2_pool = tf.nn.max_pool(h2, 
                             ksize = [1, 2, 2, 1],
                             strides = [1, 2, 2, 1], 
                             padding = 'SAME')
    
    ## 3rd layer: Fully connected
    print('\nBuilding 3rd layer:')
    h3 = fc_layer(h2_pool, name = 'fc_3',
                  n_output_units = 1024, 
                  activation_fn = tf.nn.relu)
    
    ## Dropout
    keep_prob = tf.placeholder(tf.float32, name = 'fc_keep_prob')
    h3_drop = tf.nn.dropout(h3, keep_prob = keep_prob, name = 'dropout-layer')
    
    ## 4th layer: Fully Connected (linear activation)
    print('\nBuilding 4th layer:')
    h4 = fc_layer(h3_drop, name = 'fc_4',
                  n_output_units = 10, 
                  activation_fn = None)
    
    ## Prediction
    predictions = {
        'probabilities': tf.nn.softmax(h4, name = 'probabilities'),
        'labels': tf.cast(tf.argmax(h4, axis =1), tf.int32, name = 'labels')
    }
    
    ## Visualize the graph using TensorBoard
    
    ## Loss Function and Optimisation
    cross_entropy_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits = h4, labels = tf_y_onehot),
        name = 'cross_entropy_loss')
    
    ## Optimizer:
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(cross_entropy_loss, name = 'train_op')
    
    ## Computing prediction accuracy
    correct_predictions = tf.equal(
        predictions['labels'], tf_y,
        name = 'correct_preds')
    
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name = 'accuracy')
    
def save(saver, sess, epoch, path = './model/'):
    if not os.path.isdir(path):
        os.makedirs(path)
    print('Saving model in %s' % path)
    saver.save(sess, os.path.join(path,'cnn-model.ckpt'),
               global_step = epoch)
    
def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(
            path, 'cnn-model.ckpt-%d' % epoch))
    
def train(sess, training_set, validation_set = None,
          initialize = True, epochs = 20, shuffle = True,
          dropout = 0.5, random_seed = None):

    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []

    ## initialize variables
    if initialize:
        sess.run(tf.global_variables_initializer())

    np.random.seed(random_seed) # for shuflling in batch_generator
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(
                        X_data, y_data, 
                        shuffle = shuffle)
        avg_loss = 0.0
        for i,(batch_x,batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x, 
                    'tf_y:0': batch_y, 
                    'fc_keep_prob:0': dropout}
            loss, _ = sess.run(
                    ['cross_entropy_loss:0', 'train_op'],
                    feed_dict = feed)
            avg_loss += loss

        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' % (
            epoch, avg_loss), end=' ')
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0],
                    'tf_y:0': validation_set[1],
                    'fc_keep_prob:0':1.0}
            valid_acc = sess.run('accuracy:0', feed_dict = feed)
            print(' Validation Acc: %7.3f' % valid_acc)
        else:
            print()

            
def predict(sess, X_test, return_proba = False):
    feed = {'tf_x:0': X_test, 
            'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run('probabilities:0', feed_dict = feed)
    else:
        return sess.run('labels:0', feed_dict = feed)

In [22]:
## Define random seed
random_seed = 123
np.random.seed(random_seed)

In [23]:
## create a graph
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    ## build the graph
    build_cnn()
    ## saver:
    saver = tf.train.Saver()


Building 1st layer: 
<tf.Variable 'conv1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32)
Ten

In [24]:
## create a TF session 
## and train the CNN model

with tf.Session(graph = g) as sess:
    train(sess, 
          training_set = (X_train_centered, y_train), 
          validation_set = (X_valid_centered, y_valid), 
          initialize = True,
          random_seed = 123)
    save(saver, sess, epoch = 20)

Epoch 01 Training Avg. Loss: 278.617  Validation Acc:   0.972
Epoch 02 Training Avg. Loss:  74.391  Validation Acc:   0.981
Epoch 03 Training Avg. Loss:  50.751  Validation Acc:   0.986
Epoch 04 Training Avg. Loss:  39.466  Validation Acc:   0.988
Epoch 05 Training Avg. Loss:  31.170  Validation Acc:   0.988
Epoch 06 Training Avg. Loss:  27.540  Validation Acc:   0.990
Epoch 07 Training Avg. Loss:  23.661  Validation Acc:   0.990
Epoch 08 Training Avg. Loss:  20.069  Validation Acc:   0.990
Epoch 09 Training Avg. Loss:  17.496  Validation Acc:   0.992
Epoch 10 Training Avg. Loss:  15.789  Validation Acc:   0.991
Epoch 11 Training Avg. Loss:  13.648  Validation Acc:   0.991
Epoch 12 Training Avg. Loss:  11.264  Validation Acc:   0.991
Epoch 13 Training Avg. Loss:  10.604  Validation Acc:   0.991
Epoch 14 Training Avg. Loss:   9.730  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   7.627  Validation Acc:   0.992
Epoch 16 Training Avg. Loss:   7.087  Validation Acc:   0.991
Epoch 17

In [25]:
### Calculate prediction accuracy
### on test set
### restoring the saved model

del g

## create a new graph 
## and build the model
g2 = tf.Graph()
with g2.as_default():
    tf.set_random_seed(random_seed)
    ## build the graph
    build_cnn()

    ## saver:
    saver = tf.train.Saver()

## create a new session 
## and restore the model
with tf.Session(graph = g2) as sess:
    load(saver, sess, 
         epoch = 20, path = './model/')
    
    preds = predict(sess, X_test_centered, 
                    return_proba = False)

    print('Test Accuracy: %.3f%%' % (100*
                np.sum(preds == y_test)/len(y_test)))
    


Building 1st layer: 
<tf.Variable 'conv1/_weights:0' shape=(5, 5, 1, 32) dtype=float32_ref>
<tf.Variable 'conv1/_biases:0' shape=(32,) dtype=float32_ref>
Tensor("conv1/Conv2D:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv1/net_pre-activation:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("conv1/activation:0", shape=(?, 24, 24, 32), dtype=float32)

Building 2nd layer: 
<tf.Variable 'conv_2/_weights:0' shape=(5, 5, 32, 64) dtype=float32_ref>
<tf.Variable 'conv_2/_biases:0' shape=(64,) dtype=float32_ref>
Tensor("conv_2/Conv2D:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/net_pre-activation:0", shape=(?, 8, 8, 64), dtype=float32)
Tensor("conv_2/activation:0", shape=(?, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/_weights:0' shape=(1024, 1024) dtype=float32_ref>
<tf.Variable 'fc_3/_biases:0' shape=(1024,) dtype=float32_ref>
Tensor("fc_3/MatMul:0", shape=(?, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(?, 1024), dtype=float32)
Ten

In [26]:
## run the prediction on 
##  some test samples

np.set_printoptions(precision = 2, suppress = True)

with tf.Session(graph = g2) as sess:
    load(saver, sess, 
         epoch = 20, path = './model/')
        
    print(predict(sess, X_test_centered[:10], 
              return_proba = False))
    
    print(predict(sess, X_test_centered[:10], 
                  return_proba = True))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
[7 2 1 0 4 1 4 9 5 9]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [27]:
## continue training for 20 more epochs
## without re-initializing :: initialize=False
## create a new session 
## and restore the model
with tf.Session(graph = g2) as sess:
    load(saver, sess, 
         epoch = 20, path = './model/')
    
    train(sess,
          training_set = (X_train_centered, y_train), 
          validation_set = (X_valid_centered, y_valid),
          initialize = False,
          epochs = 20,
          random_seed = 123)
        
    save(saver, sess, epoch = 40, path = './model/')
    
    preds = predict(sess, X_test_centered, 
                    return_proba = False)
    
    print('Test Accuracy: %.3f%%' % (100*
                np.sum(preds == y_test)/len(y_test)))

Loading model from ./model/
INFO:tensorflow:Restoring parameters from ./model/cnn-model.ckpt-20
Epoch 01 Training Avg. Loss:   4.409  Validation Acc:   0.992
Epoch 02 Training Avg. Loss:   4.612  Validation Acc:   0.992
Epoch 03 Training Avg. Loss:   4.052  Validation Acc:   0.991
Epoch 04 Training Avg. Loss:   2.938  Validation Acc:   0.991
Epoch 05 Training Avg. Loss:   2.942  Validation Acc:   0.993
Epoch 06 Training Avg. Loss:   3.031  Validation Acc:   0.993
Epoch 07 Training Avg. Loss:   4.060  Validation Acc:   0.992
Epoch 08 Training Avg. Loss:   2.818  Validation Acc:   0.993
Epoch 09 Training Avg. Loss:   1.650  Validation Acc:   0.993
Epoch 10 Training Avg. Loss:   2.548  Validation Acc:   0.993
Epoch 11 Training Avg. Loss:   2.288  Validation Acc:   0.993
Epoch 12 Training Avg. Loss:   2.405  Validation Acc:   0.993
Epoch 13 Training Avg. Loss:   2.367  Validation Acc:   0.993
Epoch 14 Training Avg. Loss:   1.498  Validation Acc:   0.992
Epoch 15 Training Avg. Loss:   2.431

In [33]:
with tf.Session(graph = g2) as sess:
    file_writer = tf.summary.FileWriter(logdir = './tensorboard/', graph = g2)

## Implementing CNN in Tensorflow Layers API 